In [1]:
import geemap
import ee
from datetime import date
import datetime
from ipyfilechooser import FileChooser
import pandas as pd
ee.Initialize()

In [2]:
NA_export_region = ee.Geometry.Polygon(
        [[[-113.19195907269143, 41.768481881452075],
          [-113.19195907269143, 41.21713179714162],
          [-112.44626205120706, 41.21713179714162],
          [-112.44626205120706, 41.768481881452075]]])

SA_export_region = ee.Geometry.Polygon(
        [[[-112.8838305102725, 41.2295267918564],
          [-112.8838305102725, 40.638172391988064],
          [-111.9390062915225, 40.638172391988064],
          [-111.9390062915225, 41.2295267918564]]])

def NA_Export_Clip(image):
    return image.clip(NA_export_region).copyProperties(image)
def SA_Export_Clip(image):
    return image.clip(SA_export_region).copyProperties(image)

In [3]:
### Shapefile Asset intake ###
GSL_SA_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/GSL_SouthArm_1984_Boundary")
GSL_NA_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/GSL_NorthArm_Boundary_Final")
GSL_NA_boundary_old = ee.FeatureCollection("projects/ee-radwinski/assets/GSL_NorthArm_1984_Boundary_Expanded")
GSL_total_boundary = GSL_NA_boundary.merge(GSL_SA_boundary).union()
BSF_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/BSF_Boundary_1984")
BSF_late_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/BSF_Extent_1986_Smaller")
NFB_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/NFB_Boundary_1984")
NA_SaltPool = ee.FeatureCollection("projects/ee-radwinski/assets/SaltPoolBoundaryUpdated")
NA_SaltPool_old = ee.FeatureCollection("projects/ee-radwinski/assets/GSL_NA_SaltPool_toMask")
BB_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/BonnevilleBasinShape")
BB_cropped_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/Bonneville_Basin_Cropped_Boundary")
BB_and_GSL_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/All_BB_Seds_and_Shoreline_Boundary")
BRB_boundary = ee.FeatureCollection("projects/ee-radwinski/assets/BRB_Boundary")
BRB_SaltPool = ee.FeatureCollection("projects/ee-radwinski/assets/BRB_SaltPools")
GSL_export_region = ee.Geometry.Polygon([[[-113.21733207756196, 41.80229778681049], [-113.21733207756196, 40.63715498285718], [-111.92643852287446, 40.63715498285718], [-111.92643852287446, 41.80229778681049]]])
BSF_export_region = ee.Geometry.Polygon([[[-114.10722465568696, 40.9324608917365], [-114.10722465568696, 40.55373327911588], [-113.59910697990571, 40.55373327911588], [-113.59910697990571, 40.9324608917365]]])
USGS_water_station_region = ee.Geometry.Point([-112.23, 41.066944]).buffer(500)
landsat_tile_overflow_area = ee.Geometry.Polygon([[[-111.67547275476988, 40.56441023109152], [-111.70293857508238, 41.55020817425386], [-112.08141905793187, 41.54589403254853], [-112.0778046333898, 41.32558038715473], [-112.07547209577153, 41.17039827691482], [-112.07337928992675, 41.063050532354744], [-112.07193971329473, 40.96368190627725], [-112.06982741402413, 40.83717571457464], [-112.06538696289063, 40.557904349873844]]])
farmington_bay_salt_mask_area = ee.Geometry.Polygon(
        [[[-112.08273604182838, 41.21786944789319],
          [-112.21594527034401, 41.23026430280494],
          [-112.29559614925026, 41.212704231634646],
          [-112.26401045589088, 41.16206353011265],
          [-112.25851729182838, 41.12172967661335],
          [-112.23242476253151, 41.09689651511007],
          [-112.22418501643776, 41.083441297796334],
          [-112.23517134456276, 41.05755813551418],
          [-112.22967818050026, 41.05030902417058],
          [-112.2046156194651, 41.043835928336655],
          [-112.1936292913401, 41.04176440318438],
          [-112.18195631770729, 41.032700713996185],
          [-112.17268660335182, 40.99980199072419],
          [-112.15826704768776, 40.985549413879326],
          [-112.14651167265878, 40.96831781679227],
          [-112.13930189482674, 40.95016897806219],
          [-112.13071882597909, 40.93486822434222],
          [-112.1097761379908, 40.90996450684567],
          [-112.09226667754159, 40.902439663403406],
          [-112.07819044463143, 40.899585188456534],
          [-112.0658308254908, 40.89802815019883],
          [-112.06720411650643, 40.966503157478996]]])
SA_Salt_Mine_Area = ee.Geometry.Polygon(
        [[[-112.74155702451525, 40.93565853875015],
          [-112.74696435788927, 40.93079532702177],
          [-112.74335946897325, 40.91419286803555],
          [-112.73443307737169, 40.91023619808089],
          [-112.7036198602086, 40.9169818537473],
          [-112.69872751096544, 40.91840873116959],
          [-112.70379152158556, 40.925542656271915],
          [-112.70370569089708, 40.93669597758743]]])
### Functions ###
def image_dater(image):
    date = ee.Number(image.date().format('YYYY-MM-dd'))
    return image.set({'Date_Filter': date})

def image_grab(img_col, img_selector):
    new_col = img_col.filter(ee.Filter.eq('Date_Filter', img_selector)) 
    new_col_list = new_col.toList(new_col.size())
    return ee.Image(new_col_list.get(0))

def maskL8clouds(image):
    cloudBitMask = ee.Number(2).pow(3).int()
    CirrusBitMask = ee.Number(2).pow(2).int()
    qa = image.select('QA_PIXEL')
    cloud_mask = qa.bitwiseAnd(cloudBitMask).eq(0)
    cirrus_mask = qa.bitwiseAnd(CirrusBitMask).eq(0)
    return image.updateMask(cloud_mask).updateMask(cirrus_mask)

def MaskCloudsS2(image):
  SCL = image.select('SCL')
  CloudMask = SCL.neq(9)
  return image.updateMask(CloudMask)

def MaskWaterS2(image):
  SCL = image.select('SCL')
  CloudMask = SCL.neq(6)
  return image.updateMask(CloudMask)

def landsat_ndwi_fn(image):
    ndwi_calc = image.normalizedDifference(['SR_B3', 'SR_B5']) #green-NIR / green+NIR -- full NDWI image
    standing_water = ndwi_calc.rename('ndwi').copyProperties(image) 
    return standing_water

def sentinel_ndwi_fn(image):
    ndwi_calc = image.normalizedDifference(['B3', 'B8']) #green-NIR / green+NIR -- full NDWI image
    standing_water = ndwi_calc.rename('ndwi').copyProperties(image) 
    return standing_water

landsat_ndvi_threshold = 0.105 

def landsat_ndvi_fn(image):
    ndvi_calc = image.normalizedDifference(['SR_B5', 'SR_B4']) #NIR-RED/NIR+RED -- full NDVI image
    vegetation = ndvi_calc.updateMask(ndvi_calc.gte(landsat_ndvi_threshold)).rename('ndvi').copyProperties(image) # subsets the image to just water pixels
    return vegetation

def landsat_BRB_ndvi_fn(image):
    ndvi_calc = image.normalizedDifference(['SR_B5', 'SR_B4']) #NIR-RED/NIR+RED -- full NDVI image
    vegetation = ndvi_calc.updateMask(ndvi_calc.gte(0.09)).rename('ndvi').copyProperties(image) # subsets the image to just water pixels
    return vegetation

sentinel_ndvi_threshold = 0.185 

def sentinel_ndvi_fn(image):
    ndvi_calc = image.normalizedDifference(['B8', 'B4']) #NIR-RED/NIR+RED -- full NDVI image
    vegetation = ndvi_calc.updateMask(ndvi_calc.gte(sentinel_ndvi_threshold)).rename('ndvi').copyProperties(image) # subsets the image to just water pixels
    return vegetation

def landsat_halite_fn(image):
    halite_index = image.normalizedDifference(['SR_B4', 'SR_B6']) # red-swir1 / red+swir1
    halite = halite_index.updateMask(halite_index.gte(0.1)).rename('halite').copyProperties(image)
    return halite 

landsat_halite_threshold = 0.345 

def landsat_halite_fn_conservative(image):
    halite_index = image.normalizedDifference(['SR_B4', 'SR_B6']) # red-swir1 / red+swir1
    halite = halite_index.updateMask(halite_index.gte(landsat_halite_threshold)).rename('halite').copyProperties(image)  
    return halite 

landsat_gypsum_threshold = 0.153 

def landsat_gypsum_fn(image):
    gypsum_index = image.normalizedDifference(['SR_B6', 'SR_B7'])
    gypsum = gypsum_index.updateMask(gypsum_index.gte(landsat_gypsum_threshold)).rename('gypsum').copyProperties(image)
    return gypsum

sentinel_halite_threshold = 0.58 

def sentinel_halite_fn(image):
    halite_index = image.normalizedDifference(['B4', 'B11']) # red-swir1 / red+swir1
    halite = halite_index.updateMask(halite_index.gte(sentinel_halite_threshold)).rename('halite').copyProperties(image) 
    return halite 

sentinel_gypsum_threshold = 0.3 

def sentinel_gypsum_fn(image):
    gypsum_index = image.normalizedDifference(['B11', 'B12']) # red-swir1 / red+swir1
    gypsum = gypsum_index.updateMask(gypsum_index.gte(sentinel_gypsum_threshold)).rename('gypsum').copyProperties(image) 
    return gypsum

def MaskToWaterL8(image):
  WaterBitMask = ee.Number(2).pow(7).int()
  qa = image.select('QA_PIXEL')
  water_extract = qa.bitwiseAnd(WaterBitMask).neq(0)
  return image.updateMask(water_extract).copyProperties(image)

def MaskWaterLandsat(image):
    WaterBitMask = ee.Number(2).pow(7).int()
    qa = image.select('QA_PIXEL')
    water_extract = qa.bitwiseAnd(WaterBitMask).eq(0)
    water_pixels = qa.bitwiseAnd(WaterBitMask).neq(0)
    water_count = image.select('SR_B5').rename('water_count').updateMask(water_pixels).reduceRegion(
        reducer = ee.Reducer.count(),
        geometry = BSF_late_boundary,
        scale=30)
    masked_image = image.updateMask(water_extract).copyProperties(image).set(water_count)
    return masked_image

def landsat5bandrename(img):
    return img.select('SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7', 'QA_PIXEL').rename('SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'SR_B7', 'QA_PIXEL')

def NorthArmClip(image):
    return image.clip(GSL_NA_boundary).copyProperties(image)

def SouthArmClip(image):
    return image.clip(GSL_SA_boundary).copyProperties(image)

def TotalGSLClip(image):
    return image.clip(GSL_total_boundary).copyProperties(image)

def BRBClip(image):
    return image.clip(BRB_boundary).copyProperties(image)

def MaskSaltPool(image):
    mask = ee.Image.constant(1).clip(NA_SaltPool).mask().eq(0)
    return image.updateMask(mask).copyProperties(image)

def MaskBRBSaltPool(image):
    mask = ee.Image.constant(1).clip(BRB_SaltPool).mask().eq(0)
    return image.updateMask(mask).copyProperties(image)

def MaskLandsatOverflow(image):
    mask = ee.Image.constant(1).clip(landsat_tile_overflow_area).mask().eq(0)
    return image.updateMask(mask).copyProperties(image)

def FarmingtonBaySaltMask(image):
    mask = ee.Image.constant(1).clip(farmington_bay_salt_mask_area).mask().eq(0)
    mask2 = ee.Image.constant(1).clip(SA_Salt_Mine_Area).mask().eq(0)
    return image.updateMask(mask).updateMask(mask2).copyProperties(image)

def GSLNA_WaterPixelCount(image):
    area_image = ee.Image.pixelArea()
    ndwi_calc = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('ndwi')
    histogram = ndwi_calc.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_NA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.15) 
    mask = ndwi_calc.select('ndwi').gte(threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndwi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=30,
        bestEffort = True)
    return image.set('ndwi', stats.get('ndwi')) 

def BRB_WaterPixelCount(image):
    area_image = ee.Image.pixelArea()
    ndwi_calc = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('ndwi')
    histogram = ndwi_calc.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = BRB_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.15) 
    mask = ndwi_calc.select('ndwi').gte(threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndwi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=30,
        bestEffort = True)
    return image.set('ndwi', stats.get('ndwi'))

def GSLNA_WaterPixelCount_Sent(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndwi').gte(sentinel_ndwi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndwi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=10,
        maxPixels = 1e12)
    return image.set('ndwi', stats.get('ndwi'))

def BRB_WaterPixelCount_Sent(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndwi').gte(sentinel_ndwi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndwi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=10,
        maxPixels = 1e12)
    return image.set('ndwi', stats.get('ndwi'))

def GSLSA_WaterPixelCount(image):
    area_image = ee.Image.pixelArea()
    ndwi_calc = image.normalizedDifference(['SR_B3', 'SR_B5']).rename('ndwi')
    histogram = ndwi_calc.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_SA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.15) 
    mask = ndwi_calc.select('ndwi').gte(threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndwi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=30,
        bestEffort = True)
    return image.set('ndwi', stats.get('ndwi'))

def GSLSA_WaterPixelCount_Sent(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndwi').gte(sentinel_ndwi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndwi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=10,
        maxPixels = 1e12)
    return image.set('ndwi', stats.get('ndwi'))

def GSLNA_VegPixelCount(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndvi').gte(landsat_ndvi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndvi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=30,
        maxPixels = 1e12)
    return image.set('ndvi', stats.get('ndvi'))

def BRB_VegPixelCount(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndvi').gte(landsat_ndvi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndvi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=30,
        maxPixels = 1e12)
    return image.set('ndvi', stats.get('ndvi'))

def GSLNA_VegPixelCount_Sent(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndvi').gte(sentinel_ndvi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndvi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=10,
        maxPixels = 1e12)
    return image.set('ndvi', stats.get('ndvi'))

def BRB_VegPixelCount_Sent(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndvi').gte(sentinel_ndvi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndvi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=10,
        maxPixels = 1e12)
    return image.set('ndvi', stats.get('ndvi'))

def GSLSA_VegPixelCount(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndvi').gte(landsat_ndvi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndvi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=30,
        maxPixels = 1e12)
    return image.set('ndvi', stats.get('ndvi'))

def GSLSA_VegPixelCount_Sent(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('ndvi').gte(sentinel_ndvi_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('ndvi').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=10,
        maxPixels = 1e12)
    return image.set('ndvi', stats.get('ndvi'))

def halite_pixel_count_landsat(image):
    stats = image.gt(landsat_halite_threshold).select('halite').pixelArea().rename('halite').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BSF_boundary,
        scale=30)       
    return image.set('halite', stats.get('halite'))

def halite_pixel_count_landsat_GSLNA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('halite').gte(landsat_halite_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('halite').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=30,
        maxPixels = 1e12)      
    return image.set('halite', stats.get('halite'))

def halite_pixel_count_landsat_BRB(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('halite').gte(landsat_halite_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('halite').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=30,
        maxPixels = 1e12)      
    return image.set('halite', stats.get('halite'))

def halite_pixel_count_landsat_GSLSA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('halite').gte(landsat_halite_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('halite').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('halite', stats.get('halite'))

def gypsum_pixel_count_landsat_GSLNA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('gypsum').gte(landsat_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('gypsum').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('gypsum', stats.get('gypsum'))

def gypsum_pixel_count_landsat_BRB(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('gypsum').gte(landsat_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('gypsum').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('gypsum', stats.get('gypsum'))

def gypsum_pixel_count_landsat_GSLSA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('gypsum').gte(landsat_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('gypsum').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=30,
        maxPixels = 1e12)        
    return image.set('gypsum', stats.get('gypsum'))

def carbonate_pixel_count_landsat_GSLNA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('carb_muds').lt(landsat_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('carb_muds').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('carb_muds', stats.get('carb_muds'))

def carbonate_pixel_count_landsat_BRB(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('carb_muds').lt(landsat_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('carb_muds').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('carb_muds', stats.get('carb_muds'))

def carbonate_pixel_count_landsat_GSLSA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('carb_muds').lt(landsat_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('carb_muds').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=30,
        maxPixels = 1e12)        
    return image.set('carb_muds', stats.get('carb_muds'))

def FLHV_pixel_count_landsat_GSLNA(image):
    stats = image.select('FLHV').reduceRegion(
        reducer = ee.Reducer.count(),
        geometry= GSL_NA_boundary,
        scale=30,
        maxPixels=90000000)       
    return image.set(stats)

def FLHV_pixel_count_landsat_GSLSA(image):
    stats = image.select('FLHV').reduceRegion(
        reducer = ee.Reducer.count(),
        geometry= GSL_SA_boundary,
        scale=30,
        maxPixels=90000000)       
    return image.set(stats)

def FLHV_avg_landsat_GSLNA(image):
    stats = image.select('FLHV').reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry= GSL_NA_boundary,
        scale=30,
        maxPixels=90000000)       
    return image.set(stats)

def FLHV_avg_landsat_GSLSA(image):
    stats = image.select('FLHV').reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry= GSL_SA_boundary,
        scale=30,
        maxPixels=90000000)       
    return image.set(stats)

def total_pixel_count_landsat_GSLNA(image):
    area_image = ee.Image.pixelArea()
    mask = ee.Image.constant(1).clip(NA_SaltPool).mask().eq(0)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('SumArea').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('SumArea', stats.get('SumArea'))

def total_pixel_count_landsat_BRB(image):
    area_image = ee.Image.pixelArea()
    mask = ee.Image.constant(1).mask().eq(0)
    final = image.addBands(area_image)
    stats = final.select('area').rename('SumArea').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('SumArea', stats.get('SumArea'))

def total_pixel_count_landsat_GSLSA(image):
    area_image = ee.Image.pixelArea()
    final = image.addBands(area_image)
    stats = final.select('area').rename('SumArea').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=30,
        maxPixels = 1e12)         
    return image.set('SumArea', stats.get('SumArea'))

def total_pixel_count_sentinel_GSLNA(image):
    area_image = ee.Image.pixelArea()
    mask = ee.Image.constant(1).clip(NA_SaltPool).mask().eq(0)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('SumArea').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('SumArea', stats.get('SumArea'))

def total_pixel_count_sentinel_BRB(image):
    area_image = ee.Image.pixelArea()
    mask = ee.Image.constant(1).mask().eq(0)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('SumArea').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=30,
        maxPixels = 1e12)       
    return image.set('SumArea', stats.get('SumArea'))

def total_pixel_count_sentinel_GSLSA(image):
    area_image = ee.Image.pixelArea()
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(image.select('B3').gt(0)).rename('SumArea').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=30,
        maxPixels = 1e12)         
    return image.set('SumArea', stats.get('SumArea'))

def halite_pixel_count_sent_SA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('halite').gte(sentinel_halite_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('halite').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=10,
        maxPixels = 1e12)      
    return image.set('halite', stats.get('halite'))

def halite_pixel_count_sent_NA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('halite').gte(sentinel_halite_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('halite').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=10,
        maxPixels = 1e12)       
    return image.set('halite', stats.get('halite'))

def halite_pixel_count_sent_BRB(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('halite').gte(sentinel_halite_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('halite').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=10,
        maxPixels = 1e12)       
    return image.set('halite', stats.get('halite'))

def carbonate_pixel_count_sent_SA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('carb_muds').lt(sentinel_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('carb_muds').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=10,
        maxPixels = 1e12)        
    return image.set('carb_muds', stats.get('carb_muds'))

def carbonate_pixel_count_sent_NA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('carb_muds').lt(sentinel_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('carb_muds').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=10,
        maxPixels = 1e12)      
    return image.set('carb_muds', stats.get('carb_muds'))

def carbonate_pixel_count_sent_BRB(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('carb_muds').lt(sentinel_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('carb_muds').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=10,
        maxPixels = 1e12)      
    return image.set('carb_muds', stats.get('carb_muds'))

def gypsum_pixel_count_sent_SA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('gypsum').gte(sentinel_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('gypsum').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_SA_boundary,
        scale=10,
        maxPixels = 1e12)      
    return image.set('gypsum', stats.get('gypsum'))

def gypsum_pixel_count_sent_NA(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('gypsum').gte(sentinel_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('gypsum').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= GSL_NA_boundary,
        scale=10,
        maxPixels = 1e12)       
    return image.set('gypsum', stats.get('gypsum'))

def gypsum_pixel_count_sent_BRB(image):
    area_image = ee.Image.pixelArea()
    mask = image.select('gypsum').gte(sentinel_gypsum_threshold)
    final = image.addBands(area_image)
    stats = final.select('area').updateMask(mask).rename('gypsum').reduceRegion(
        reducer = ee.Reducer.sum(),
        geometry= BRB_boundary,
        scale=10,
        maxPixels = 1e12)       
    return image.set('gypsum', stats.get('gypsum'))

def BDA_pixel_count_sent_SA(image):
    stats = image.select('2BDA').reduceRegion(
        reducer = ee.Reducer.count(),
        geometry= GSL_SA_boundary,
        scale=20,
        maxPixels = 1e12)       
    return image.set(stats)

def BDA_pixel_count_sent_NA(image):
    stats = image.select('2BDA').reduceRegion(
        reducer = ee.Reducer.count(),
        geometry= GSL_NA_boundary,
        scale=20,
        maxPixels = 1e12)       
    return image.set(stats)

def BDA_average_sent_NA(image):
    stats = image.select('2BDA').reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry= GSL_NA_boundary,
        scale=20,
        maxPixels = 1e12)       
    return image.set(stats)

def BDA_average_sent_SA(image):
    stats = image.select('2BDA').reduceRegion(
        reducer = ee.Reducer.mean(),
        geometry= GSL_SA_boundary,
        scale=20,
        maxPixels = 1e12)       
    return image.set(stats)

def OtsuThreshold(histogram):
    counts = ee.Array(ee.Dictionary(histogram).get('histogram'))
    means = ee.Array(ee.Dictionary(histogram).get('bucketMeans'))
    size = means.length().get([0])
    total = counts.reduce(ee.Reducer.sum(), [0]).get([0])
    sum = means.multiply(counts).reduce(ee.Reducer.sum(), [0]).get([0])
    mean = sum.divide(total)
    indices = ee.List.sequence(1, size)

    def func_xxx(i):
        aCounts = counts.slice(0, 0, i)
        aCount = aCounts.reduce(ee.Reducer.sum(), [0]).get([0])
        aMeans = means.slice(0, 0, i)
        aMean = (
            aMeans.multiply(aCounts)
            .reduce(ee.Reducer.sum(), [0])
            .get([0])
            .divide(aCount)
        )
        bCount = total.subtract(aCount)
        bMean = sum.subtract(aCount.multiply(aMean)).divide(bCount)
        return aCount.multiply(aMean.subtract(mean).pow(2)).add(
            bCount.multiply(bMean.subtract(mean).pow(2)))

    bss = indices.map(func_xxx)
    return means.sort(bss).get([-1])

def extract_water_NA_ls(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_NA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.15) 
    water = image.select('ndwi').updateMask(image.select('ndwi').gte(threshold)).rename('ndwi').copyProperties(image)
    return water.set({"threshold": threshold}).copyProperties(image)

def extract_water_SA_ls(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_SA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.175)
    water = image.select('ndwi').updateMask(image.select('ndwi').gte(threshold)).rename('ndwi').copyProperties(image)
    return water.set({"threshold": threshold})

def extract_water_BRB_ls(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = BRB_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.175) 
    water = image.select('ndwi').updateMask(image.select('ndwi').gte(threshold)).rename('ndwi').copyProperties(image)
    return water.set({"threshold": threshold})

def GSL_NA_halite_watermask(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_NA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.15) 
    mask = image.select('ndwi').lte(threshold)
    return image.updateMask(mask)

def BRB_halite_watermask(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = BRB_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.175) 
    mask = image.select('ndwi').lte(threshold)
    return image.updateMask(mask)

def GSL_SA_halite_watermask(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_SA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.175)
    mask = image.select('ndwi').lte(threshold)
    return image.updateMask(mask)

sentinel_ndwi_threshold = 0.06

def GSL_S2_watermask(image):
    mask = image.select('ndwi').lte(sentinel_ndwi_threshold) 
    return image.updateMask(mask)    

def GSL_SA_S2_watermask(image):
    mask = image.select('ndwi').lte(0.04) 
    return image.updateMask(mask) 

def BRB_S2_watermask(image):
    mask = image.select('ndwi').lte(0.04) 
    return image.updateMask(mask)

def extract_water_NA_S2(image):
 
    water = image.select('ndwi').updateMask(image.select('ndwi').gte(sentinel_ndwi_threshold)).rename('ndwi').copyProperties(image) 
    return water

def extract_water_SA_S2(image):

    water = image.select('ndwi').updateMask(image.select('ndwi').gte(sentinel_ndwi_threshold)).rename('ndwi').copyProperties(image) 
    return water

def extract_water_BRB_S2(image):

    water = image.select('ndwi').updateMask(image.select('ndwi').gte(sentinel_ndwi_threshold)).rename('ndwi').copyProperties(image) 
    return water

def CollectionStitch(col1, col2, dates_list): #this function mosaics north and south images only if their dates match. Ignores scenes without a partner.
    image_list = []
    for date in dates_list:
        if date in col1.aggregate_array('Date_Filter').getInfo() and date in col2.aggregate_array('Date_Filter').getInfo(): #conditional statement to ensure this will only run if there are two images with matching dates. Avoids errors
            filtered_col1 = image_grab(col1, date)
            filtered_col2 = image_grab(col2, date)
            merged_col = ee.ImageCollection.fromImages([filtered_col1, filtered_col2])
            mosaic = merged_col.mosaic().copyProperties(filtered_col1) #new collection images contain all image properties of the northern landsat image
            image_list.append(mosaic)
        else: None #If the condition isnt met, do nothing and keep going through the list
    new_col = ee.ImageCollection.fromImages(image_list)
    return new_col

def temperature_bands(img):
    scale1 = ['ST_ATRAN', 'ST_EMIS']
    scale2 = ['ST_DRAD', 'ST_TRAD', 'ST_URAD']
    scale1_names = ['transmittance', 'emissivity']
    scale2_names = ['downwelling', 'B10_radiance', 'upwelling']
    scale1_bands = img.select(scale1).multiply(0.0001).rename(scale1_names) #Scaled to new L8 collection
    scale2_bands = img.select(scale2).multiply(0.001).rename(scale2_names) #Scaled to new L8 collection
    return img.addBands(scale1_bands).addBands(scale2_bands).copyProperties(img)

def landsat_LST(image):
    # Based on Sekertekin, A., & Bonafoni, S. (2020) https://doi.org/10.3390/rs12020294
    
    k1 = 774.89
    k2 = 1321.08
    LST = image.expression(
        '(k2/log((k1/((B10_rad - upwelling - transmittance*(1 - emissivity)*downwelling)/(transmittance*emissivity)))+1)) - 273.15',
        {'k1': k1,
        'k2': k2,
        'B10_rad': image.select('B10_radiance'),
        'upwelling': image.select('upwelling'),
        'transmittance': image.select('transmittance'),
        'emissivity': image.select('emissivity'),
        'downwelling': image.select('downwelling')}).rename('LST')
    return image.addBands(LST).copyProperties(image) 

def average_GSL_NA_temperature(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_NA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.15) 
    water = image.select('ndwi').updateMask(image.select('ndwi').gte(threshold))
    stats = image.select('LST').updateMask(image.select('ndwi').gte(threshold)).reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = GSL_NA_boundary,
    scale=30)  
    return image.updateMask(image.select('ndwi').gte(threshold)).copyProperties(image).set(stats) 

def average_GSL_SA_temperature(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_SA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.175) 
    water = image.select('ndwi').updateMask(image.select('ndwi').gte(threshold)) 
    stats = image.select('LST').updateMask(image.select('ndwi').gte(threshold)).reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = GSL_SA_boundary,
    scale=30)  
    return image.updateMask(image.select('ndwi').gte(threshold)).copyProperties(image).set(stats)
def average_GSL_waterstation_temperature(image):
    histogram = image.select('ndwi').reduceRegion(
        reducer = ee.Reducer.histogram(255, 2),
        geometry = GSL_SA_boundary.geometry().buffer(6000),
        scale = 30,
        bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.175)
    water = image.select('ndwi').updateMask(image.select('ndwi').gte(threshold)) 
    stats = image.select('LST').updateMask(image.select('ndwi').gte(threshold)).reduceRegion(
    reducer = ee.Reducer.mean(),
    geometry = USGS_water_station_region,
    scale=30)  
    return image.set(stats).updateMask(image.select('ndwi').gte(threshold))

def MaskToCarbonatesL8(image):
    gypsum = image.select('gypsum')
    halite = image.select('halite')
    halite_index = image.normalizedDifference(['SR_B4', 'SR_B6'])
    gypsum_index = image.normalizedDifference(['SR_B6', 'SR_B7'])
    mask1 = gypsum_index.updateMask(halite_index.lt(landsat_halite_threshold)).updateMask(gypsum_index.lt(landsat_gypsum_threshold)).rename('carb_muds').copyProperties(image)
    return mask1

def MaskToCarbonatesS2(image):
    halite_index = image.normalizedDifference(['B4', 'B11'])
    gypsum_index = image.normalizedDifference(['B11', 'B12'])
    mask1 = gypsum_index.updateMask(halite_index.lt(sentinel_halite_threshold)).updateMask(gypsum_index.lt(sentinel_gypsum_threshold)).rename('carb_muds').copyProperties(image)
    return mask1

############# CHLOROPHYLL SPECTRAL INDICES #############

### Sentinel 2 Band Algorithm adapted from Dall’Olmo, G., & Gitelson, A. A. (2006). Effect of bio-optical parameter variability and uncertainties 
#in reflectance measurements on the remote estimation of chlorophyll-a concentration in turbid productive waters: 
# Modeling results. Applied Optics, 45(15), 3577. https://doi.org/10.1364/AO.45.003577
def landsat_MaskToWater_NA(image):
    histogram = image.select('ndwi').reduceRegion(
    reducer = ee.Reducer.histogram(255, 2),
    geometry = GSL_NA_boundary.geometry().buffer(6000),
    scale = 30,
    bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.15) 
    mask = image.select('ndwi').gte(threshold)
    return image.updateMask(mask).copyProperties(image)

def landsat_MaskToWater_SA(image):
    histogram = image.select('ndwi').reduceRegion(
    reducer = ee.Reducer.histogram(255, 2),
    geometry = GSL_SA_boundary.geometry().buffer(6000),
    scale = 30,
    bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.175) 
    mask = image.select('ndwi').gte(threshold)
    return image.updateMask(mask).copyProperties(image)

def landsat_MaskToWater_BRB(image):
    histogram = image.select('ndwi').reduceRegion(
    reducer = ee.Reducer.histogram(255, 2),
    geometry = BRB_boundary.geometry().buffer(6000),
    scale = 30,
    bestEffort= True,)
    threshold = OtsuThreshold(histogram.get('ndwi')).add(0.175) 
    mask = image.select('ndwi').gte(threshold)
    return image.updateMask(mask).copyProperties(image)

def sentinel_MaskToWater(image):
    water = image.updateMask(image.select('ndwi').gte(sentinel_ndwi_threshold)).copyProperties(image) 
    return water

def landsat_FLHV(image):
    FLHV = image.expression(
    '(GREEN - (RED + (CA-RED)))', {
        'CA': image.select('SR_B1'),
        'GREEN': image.select('SR_B3'),
        'RED': image.select('SR_B4')})
    mask = FLHV.updateMask(FLHV.gte(3500)).rename('FLHV').copyProperties(image)
    return mask

def landsat_KIVU(image):
    KIVU = image.expression(
    '(BLUE - RED) / GREEN', {
        'BLUE': image.select('SR_B2'),
        'GREEN': image.select('SR_B3'),
        'RED': image.select('SR_B4')})
    mask = KIVU.updateMask(KIVU.gte(-1)).rename('FLHV').copyProperties(image) 
    return mask

def sentinel_2BDA(image):
    S2BDA = image.expression(
    '(RED_EDGE / RED)', {
        'RED': image.select('B4'),
        'RED_EDGE': image.select('B5')})
    mask = S2BDA.updateMask(S2BDA.gte(-1)).rename('2BDA').copyProperties(image)
    return mask

def sentinel_veg_mask(image):
    ndvi_calc = image.normalizedDifference(['B8', 'B4']) #NIR-RED/NIR+RED -- full NDVI image
    mask = image.updateMask(ndvi_calc.lt(sentinel_ndvi_threshold)).copyProperties(image) 
    return mask

def landsat_veg_mask(image):
    ndvi_calc = image.normalizedDifference(['SR_B5', 'SR_B4']) #NIR-RED/NIR+RED -- full NDVI image
    mask = image.updateMask(ndvi_calc.lt(landsat_ndvi_threshold)).copyProperties(image) 
    return mask

def landsat_halite_mask(image):
    halite_index = image.normalizedDifference(['SR_B4', 'SR_B6']) # red-swir1 / red+swir1
    mask = image.updateMask(halite_index.lt(landsat_halite_threshold)).copyProperties(image) 
    return mask 

def sentinel_halite_mask(image):
    halite_index = image.normalizedDifference(['B4', 'B11']) # red-swir1 / red+swir1
    mask = image.updateMask(halite_index.lt(sentinel_halite_threshold)).copyProperties(image) 
    return mask 

In [4]:
### Tile definition ###
#Great Salt Lake
SLC_S_tile_N = '12TUM' #sentinel
SLC_S_tile_S = '12TUL' #sentinel
SLC_L_N_row = 31 #landsat
SLC_L_S_row = 32 #landsat
SLC_L_path = 38 #landsat
# Bear River Bay
BRB_tile_row = 31
BRB_tile_path = 38
#Bonneville basin
BSF_L_path = 39 #landsat
BSF_L_row_N = 31 #landsat
BSF_L_row_S = 32 #landsat
BSF_S_tile_N = '12TTL' #sentinel
BSF_S_tile_S = '12TTK' #sentinel


In [19]:
### Landsat dataset definition and filtering ###
#Landsat 5 definition and band renaming
landsat5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2").map(landsat5bandrename)

# Note: used a sort function to sort all images to be in chronological order. Otherwise defaults to L8 images first, then L9 images out of sort.
start_date = ee.Date(str('2022-11-20'))
end_date = ee.Date(str('2023-05-30')) #date.today()))
### Great Salt Lake ###
#Landsat 5, 8, & 9 -- spanning from 1984-3-16 to 2012-05-05 and 2013-03-18 to present
GSL_landsat_N = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")).merge(landsat5).filterDate(start_date \
    .format('YYYY-MM-dd'), end_date.format('YYYY-MM-dd')).filter(ee.Filter.And(ee.Filter.eq('WRS_PATH', SLC_L_path), \
        ee.Filter.eq('WRS_ROW', SLC_L_N_row))).filter(ee.Filter.lte('CLOUD_COVER', 10)).map(image_dater).sort('Date_Filter') #.map(maskL8clouds)

GSL_landsat_S = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")).merge(landsat5).filterDate(start_date \
    .format('YYYY-MM-dd'), end_date.format('YYYY-MM-dd')).filter(ee.Filter.And(ee.Filter.eq('WRS_PATH', SLC_L_path), \
        ee.Filter.eq('WRS_ROW', SLC_L_S_row))).filter(ee.Filter.lte('CLOUD_COVER', 10)).map(image_dater).sort('Date_Filter') #.map(maskL8clouds)

GSL_landsat_dates_N = GSL_landsat_N.aggregate_array('Date_Filter').getInfo()
GSL_landsat_dates_S = GSL_landsat_S.aggregate_array('Date_Filter').getInfo()
GSL_landsat_MR = GSL_landsat_N.aggregate_array('Date_Filter').getInfo()[-1] #This is the date of image we want to grab


# bear river bay
BRB_landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")).merge(landsat5).filterDate(start_date \
    .format('YYYY-MM-dd'), end_date.format('YYYY-MM-dd')).filter(ee.Filter.And(ee.Filter.eq('WRS_PATH', BRB_tile_path), \
        ee.Filter.eq('WRS_ROW', BRB_tile_row))).filter(ee.Filter.lte('CLOUD_COVER', 10)).map(image_dater).sort('Date_Filter') #.map(maskL8clouds)

BRB_landsat_dates = BRB_landsat.aggregate_array('Date_Filter').getInfo()
BRB_landsat_MR = BRB_landsat.aggregate_array('Date_Filter').getInfo()[-1] #This is the date of image we want to grab

#Standard cloud cover has been 10%


### Bonneville Salt Flats ###
BSF_landsat_N = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")).merge(landsat5).filterDate(start_date \
    .format('YYYY-MM-dd'), end_date.format('YYYY-MM-dd')).filter(ee.Filter.And(ee.Filter.eq('WRS_PATH', BSF_L_path), \
        ee.Filter.eq('WRS_ROW', BSF_L_row_N))).filter(ee.Filter.lte('CLOUD_COVER', 10)).map(image_dater).sort('Date_Filter')

BSF_landsat_S = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").merge(ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")).merge(landsat5).filterDate(start_date \
    .format('YYYY-MM-dd'), end_date.format('YYYY-MM-dd')).filter(ee.Filter.And(ee.Filter.eq('WRS_PATH', BSF_L_path), \
        ee.Filter.eq('WRS_ROW', BSF_L_row_S))).filter(ee.Filter.lte('CLOUD_COVER', 10)).map(image_dater).sort('Date_Filter')

BSF_landsat_dates_N = BSF_landsat_N.aggregate_array('Date_Filter').getInfo()
BSF_landsat_dates_S = BSF_landsat_S.aggregate_array('Date_Filter').getInfo()
BSF_landsat_MR = BSF_landsat_N.aggregate_array('Date_Filter').getInfo()[-1]

In [20]:
landsat_GSL_mosaic_col = CollectionStitch(GSL_landsat_N, GSL_landsat_S, GSL_landsat_dates_S)
landsat_GSL_NA_mosaic_col = CollectionStitch(BSF_landsat_N, BSF_landsat_S, BSF_landsat_dates_N)

landsat_GSL_mosaic_col_dates = landsat_GSL_mosaic_col.aggregate_array('Date_Filter').getInfo() #dates to use for visualization and to pair with each observation
landsat_GSL_NA_mosaic_col_dates = landsat_GSL_NA_mosaic_col.aggregate_array('Date_Filter').getInfo()

In [26]:
### total GSL area collections for reference ###
GSL_NA_area = landsat_GSL_NA_mosaic_col.map(MaskSaltPool).map(NorthArmClip).map(total_pixel_count_landsat_GSLNA) #.map(MaskSaltPool)
GSL_SA_area = landsat_GSL_mosaic_col.map(SouthArmClip).map(total_pixel_count_landsat_GSLSA)
BRB_area = BRB_landsat.map(BRBClip).map(total_pixel_count_landsat_BRB)

[2042477248.3124743, 2042477248.3124743, 2042477248.3124743, 2042477248.3124743, 2042477248.3124743, 2042477248.3124743, 2042477248.3124745, 2042477248.3124743, 2042477248.3124743, 2042477248.3124745, 2042477248.3124743]
[1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.2959232, 1154526904.295923, 1154526904.295923, 1154526904.295923, 1154526904.295923]
1986-10-05


In [27]:
### Sentinel dataset definition and filtering ###
start_date_s = ee.Date(str('2022-11-22'))
end_date_s = ee.Date(str('2023-05-30')) 
#Great Salt Lake#
#No data pixels of 10 and clouds of 50 set to parameters used for data collection. 
GSL_sentinel_N = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.inList('MGRS_TILE', [SLC_S_tile_N])).filterDate(start_date_s.format('YYYY-MM-dd'), end_date_s.format('YYYY-MM-dd')).map(image_dater) \
    .select(['B12','B11', 'B8', 'B5', 'B4', 'B3', 'B2', 'QA60', 'SCL']).filter(ee.Filter.lte('NODATA_PIXEL_PERCENTAGE', 10)).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10)) #.filter(ee.Filter.lte('NODATA_PIXEL_PERCENTAGE', 10)).filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 20))

GSL_sentinel_S = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.inList('MGRS_TILE', [SLC_S_tile_S])).filterDate(start_date_s.format('YYYY-MM-dd'), end_date_s.format('YYYY-MM-dd')).map(image_dater) \
    .select(['B12', 'B11', 'B8', 'B5', 'B4', 'B3', 'B2', 'QA60', 'SCL']).filter(ee.Filter.lte('NODATA_PIXEL_PERCENTAGE', 10)) #.filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', 10)) #cloud filter provides too little images at lte to 5, so setting to 10
GSL_sent_dates_N = GSL_sentinel_N.aggregate_array('Date_Filter').getInfo()
GSL_sent_dates_S = GSL_sentinel_S.aggregate_array('Date_Filter').getInfo()

In [28]:
### Stitched sentinel collections ###
sentinel_GSL_mosaic_col = CollectionStitch(GSL_sentinel_N, GSL_sentinel_S, GSL_sent_dates_N)

sentinel_GSL_mosaic_col_dates = sentinel_GSL_mosaic_col.aggregate_array('Date_Filter').getInfo()

In [32]:
### total GSL area collections for reference ###
GSL_NA_area_S2 = sentinel_GSL_mosaic_col.map(NorthArmClip).map(total_pixel_count_sentinel_GSLNA)
GSL_NA_area_S2_mask = sentinel_GSL_mosaic_col.map(NorthArmClip).map(MaskSaltPool).map(total_pixel_count_sentinel_GSLNA)
GSL_SA_area_S2 = sentinel_GSL_mosaic_col.map(SouthArmClip).map(total_pixel_count_sentinel_GSLSA)

2042477248.3124745
2042477248.3124745
2636256360.8259325


In [21]:
### NDWI and LST collections ###
### Landsat
GSL_SA_NDWI = landsat_GSL_mosaic_col.map(GSLSA_WaterPixelCount).map(landsat_ndwi_fn).map(extract_water_SA_ls).map(SouthArmClip)
GSL_NA_NDWI_pre93 = landsat_GSL_NA_mosaic_col.filter(ee.Filter.lt('Date_Filter', '1994-01-01')).map(GSLNA_WaterPixelCount).map(landsat_ndwi_fn).map(extract_water_NA_ls).map(NorthArmClip) #.map(MaskSaltPool).map(GSLNA_WaterPixelCount)
GSL_NA_NDWI_post93 = landsat_GSL_NA_mosaic_col.filter(ee.Filter.gt('Date_Filter', '1994-01-01')).map(MaskSaltPool).map(GSLNA_WaterPixelCount).map(landsat_ndwi_fn).map(extract_water_NA_ls).map(NorthArmClip) #.map(MaskSaltPool).map(GSLNA_WaterPixelCount)
GSL_NA_NDWI = GSL_NA_NDWI_pre93.merge(GSL_NA_NDWI_post93).sort('Date_Filter')
BRB_NDWI = BRB_landsat.map(BRB_WaterPixelCount).map(landsat_ndwi_fn).map(extract_water_BRB_ls).map(BRBClip)

unclipped_NA_NDWI = landsat_GSL_NA_mosaic_col.map(landsat_ndwi_fn).map(MaskSaltPool)
unclipped_SA_NDWI = landsat_GSL_mosaic_col.map(landsat_ndwi_fn)
unclipped_BRB_NDWI = BRB_landsat.map(landsat_ndwi_fn)

toyFilter = ee.Filter.equals(leftField='Date_Filter', rightField='Date_Filter')
innerJoin = ee.Join.inner()
def joinedCol(image):
    return ee.Image.cat(image.get('primary'), image.get('secondary'))

combined_NA_NDWI_col = ee.ImageCollection(innerJoin.apply(GSL_NA_NDWI, landsat_GSL_NA_mosaic_col, toyFilter).map(joinedCol))
combined_NA_NDWI_co_unclipped = ee.ImageCollection(innerJoin.apply(unclipped_NA_NDWI, landsat_GSL_NA_mosaic_col, toyFilter).map(joinedCol))

GSL_NA_LST = combined_NA_NDWI_co_unclipped.map(temperature_bands).map(MaskSaltPool).map(landsat_LST).map(average_GSL_NA_temperature).map(NorthArmClip)

#merging NDWI col with normal RGB col to use for clipping image when making LST map
combined_SA_NDWI_col = ee.ImageCollection(innerJoin.apply(GSL_SA_NDWI, landsat_GSL_mosaic_col, toyFilter).map(joinedCol))
combined_SA_NDWI_co_unclipped = ee.ImageCollection(innerJoin.apply(unclipped_SA_NDWI, landsat_GSL_mosaic_col, toyFilter).map(joinedCol))

GSL_SA_LST = combined_SA_NDWI_co_unclipped.map(temperature_bands).map(landsat_LST).map(average_GSL_SA_temperature).map(SouthArmClip)
GSL_SA_USGS_LST = combined_SA_NDWI_col.map(temperature_bands).map(landsat_LST).map(average_GSL_waterstation_temperature)

combined_BRB_NDWI_col = ee.ImageCollection(innerJoin.apply(BRB_NDWI, BRB_landsat, toyFilter).map(joinedCol))
combined_BRB_NDWI_col_unclipped = ee.ImageCollection(innerJoin.apply(unclipped_BRB_NDWI, BRB_landsat, toyFilter).map(joinedCol))

### NDWI stats dataframes ###
# these dataframes contain the water area and associated date and are to be exported #

landsat_GSLNA_ndwi_df = pd.merge(pd.DataFrame({'NA_Water_Area': GSL_NA_NDWI.aggregate_array('ndwi').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer') #new
landsat_GSLSA_ndwi_df = pd.merge(pd.DataFrame({'SA_Water_Area': GSL_SA_NDWI.aggregate_array('ndwi').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')

landsat_BRB_ndwi_df = pd.merge(pd.DataFrame({'BRB_Water_Area': BRB_NDWI.aggregate_array('ndwi').getInfo()}),
                                                pd.DataFrame({'Date': BRB_landsat_dates}), right_index=True, left_index=True, how='outer') #new


landsat_GSLNA_LST_df = pd.merge(pd.DataFrame({'Temperature (C)': GSL_NA_LST.aggregate_array('LST').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer')
landsat_GSLSA_LST_df = pd.merge(pd.DataFrame({'Temperature (C)': GSL_SA_LST.aggregate_array('LST').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')
landsat_GSL_USGS_LST_df = pd.merge(pd.DataFrame({'Temperature (C)': GSL_SA_USGS_LST.aggregate_array('LST').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')

##Other useful dataframes!!
landsat_GSL_NA_NDWI_threshold_df = pd.merge(pd.DataFrame({'Threshold': GSL_NA_NDWI.aggregate_array('threshold').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer')
landsat_GSL_NA_cloud_cover_df = pd.merge(pd.DataFrame({'Threshold': GSL_NA_NDWI.aggregate_array('CLOUD_COVER').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer')
landsat_GSL_SA_NDWI_threshold_df = pd.merge(pd.DataFrame({'Threshold': GSL_SA_NDWI.aggregate_array('threshold').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')
landsat_GSL_SA_cloud_cover_df = pd.merge(pd.DataFrame({'Threshold': GSL_SA_NDWI.aggregate_array('CLOUD_COVER').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')


landsat_BRB_NDWI_threshold_df = pd.merge(pd.DataFrame({'Threshold': BRB_NDWI.aggregate_array('threshold').getInfo()}),
                                                pd.DataFrame({'Date': BRB_landsat_dates}), right_index=True, left_index=True, how='outer')

In [29]:
### NDWI collections ###
### Sentinel
sent_GSL_SA_NDWI = sentinel_GSL_mosaic_col.map(sentinel_ndwi_fn).map(extract_water_SA_S2).map(SouthArmClip).map(GSLSA_WaterPixelCount_Sent) 
sent_GSL_NA_NDWI = sentinel_GSL_mosaic_col.map(sentinel_ndwi_fn).map(extract_water_NA_S2).map(MaskSaltPool).map(NorthArmClip).map(GSLNA_WaterPixelCount_Sent) 

sent_GSL_NDWI_unclipped = sentinel_GSL_mosaic_col.combine(sentinel_GSL_mosaic_col.map(sentinel_ndwi_fn))

### NDWI stats dataframes ###
# these dataframes contain the water area and associated date and are to be exported #

sentinel_GSLNA_ndwi_df = pd.merge(pd.DataFrame({'NA_Water_Area': sent_GSL_NA_NDWI.aggregate_array('ndwi').getInfo()}),
                                                pd.DataFrame({'Date': sent_GSL_NA_NDWI.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer') 
sentinel_GSLSA_ndwi_df = pd.merge(pd.DataFrame({'SA_Water_Area': sent_GSL_SA_NDWI.aggregate_array('ndwi').getInfo()}),
                                                pd.DataFrame({'Date': sent_GSL_SA_NDWI.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer') 

sentinel_GSL_NA_cloud_cover_df = pd.merge(pd.DataFrame({'%_Cloudy_Pixels': GSL_sentinel_N.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()}),
                                                pd.DataFrame({'Date': sentinel_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')
sentinel_GSL_SA_cloud_cover_df = pd.merge(pd.DataFrame({'%_Cloudy_Pixels': GSL_sentinel_S.aggregate_array('CLOUDY_PIXEL_PERCENTAGE').getInfo()}),
                                                pd.DataFrame({'Date': GSL_sent_dates_S}), right_index=True, left_index=True, how='outer')

In [22]:
### NDVI collections ###
### Landsat
GSL_SA_NDVI = landsat_GSL_mosaic_col.map(landsat_ndvi_fn).map(SouthArmClip).map(GSLSA_VegPixelCount)
GSL_SA_NDVI_S2Area = landsat_GSL_mosaic_col.map(landsat_ndvi_fn).map(MaskLandsatOverflow).map(SouthArmClip).map(GSLSA_VegPixelCount)
GSL_NA_NDVI = landsat_GSL_NA_mosaic_col.map(MaskSaltPool).map(landsat_ndvi_fn).map(NorthArmClip).map(GSLNA_VegPixelCount)

BRB_NDVI = BRB_landsat.map(landsat_ndvi_fn).map(BRBClip).map(BRB_VegPixelCount)

### NDVI stats dataframes ###
# these dataframes contain the vegetation area and associated date and are to be exported #
#Landsat

landsat_GSLNA_ndvi_df = pd.merge(pd.DataFrame({'NA_Veg_Area': GSL_NA_NDVI.aggregate_array('ndvi').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer') #Take the counted number of pixels from ndwi image and multiply by 900 square meters, which is the area per pixel. Join with date column dataframe
landsat_GSLSA_ndvi_df = pd.merge(pd.DataFrame({'SA_Veg_Area': GSL_SA_NDVI.aggregate_array('ndvi').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer') #Take the counted number of pixels from ndwi image and multiply by 900 square meters, which is the area per pixel. Then join with date column dataframe
landsat_GSLSA_ndvi_S2Area_df = pd.merge(pd.DataFrame({'SA_Veg_Area': GSL_SA_NDVI_S2Area.aggregate_array('ndvi').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')

# Landsat
landsat_BRB_ndvi_df = pd.merge(pd.DataFrame({'NA_Veg_Area': BRB_NDVI.aggregate_array('ndvi').getInfo()}),
                                                pd.DataFrame({'Date': BRB_landsat_dates}), right_index=True, left_index=True, how='outer') #Take the counted number of pixels from ndwi image and multiply by 900 square meters, which is the area per pixel. Join with date column dataframe

In [30]:
### NDVI collections ###
### Sentinel
sent_GSL_SA_NDVI = sentinel_GSL_mosaic_col.map(sentinel_ndvi_fn).map(MaskSaltPool).map(SouthArmClip).map(GSLSA_VegPixelCount_Sent)
sent_GSL_NA_NDVI = sentinel_GSL_mosaic_col.map(sentinel_ndvi_fn).map(MaskSaltPool).map(NorthArmClip).map(GSLNA_VegPixelCount_Sent)

### NDVI stats dataframes ###
# these dataframes contain the vegetation area and associated date and are to be exported #

sent_GSLNA_ndvi_df = pd.merge(pd.DataFrame({'NA_Veg_Area': sent_GSL_NA_NDVI.aggregate_array('ndvi').getInfo()}),
                                                pd.DataFrame({'Date': sent_GSL_NA_NDVI.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer') 
sent_GSLSA_ndvi_df = pd.merge(pd.DataFrame({'SA_Veg_Area': sent_GSL_SA_NDVI.aggregate_array('ndvi').getInfo()}),
                                                pd.DataFrame({'Date': sent_GSL_SA_NDVI.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer') 

In [23]:
### Halite index collections ###
### Landsat
#GSl shoreline
GSL_NA_halite = combined_NA_NDWI_co_unclipped.map(GSL_NA_halite_watermask).map(MaskSaltPool).map(landsat_veg_mask).map(landsat_halite_fn_conservative).map(NorthArmClip).map(halite_pixel_count_landsat_GSLNA) 
GSL_SA_halite = combined_SA_NDWI_co_unclipped.map(GSL_SA_halite_watermask).map(landsat_veg_mask).map(FarmingtonBaySaltMask).map(landsat_halite_fn_conservative).map(SouthArmClip).map(halite_pixel_count_landsat_GSLSA)
BRB_halite = combined_BRB_NDWI_col_unclipped.map(BRB_halite_watermask).map(landsat_veg_mask).map(MaskBRBSaltPool).map(landsat_halite_fn_conservative).map(BRBClip).map(halite_pixel_count_landsat_BRB)

### Halite stats datatframe ###

landsat_halite_GSL_NA_df = pd.merge(pd.DataFrame({'Halite_Area': GSL_NA_halite.aggregate_array('halite').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer') 
landsat_halite_GSL_SA_df = pd.merge(pd.DataFrame({'Halite_Area': GSL_SA_halite.aggregate_array('halite').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')   
landsat_halite_BRB_df = pd.merge(pd.DataFrame({'Halite_Area': BRB_halite.aggregate_array('halite').getInfo()}),
                                                pd.DataFrame({'Date': BRB_landsat_dates}), right_index=True, left_index=True, how='outer') 

In [31]:
### Halite index collections ###
### Sentinel
GSL_halite_index_sentinel = sent_GSL_NDWI_unclipped.map(GSL_S2_watermask).map(FarmingtonBaySaltMask).map(sentinel_veg_mask).map(MaskSaltPool).map(sentinel_halite_fn).map(TotalGSLClip)
GSL_SA_halite_sent = GSL_halite_index_sentinel.map(halite_pixel_count_sent_SA)
GSL_NA_halite_sent = GSL_halite_index_sentinel.map(halite_pixel_count_sent_NA)

### Halite stats dataframes ###

sentinel_SA_halite_df = pd.merge(pd.DataFrame({'Halite_Area': GSL_SA_halite_sent.aggregate_array('halite').getInfo()}),
                                                pd.DataFrame({'Date': GSL_SA_halite_sent.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer')
sentinel_NA_halite_df = pd.merge(pd.DataFrame({'Halite_Area': GSL_NA_halite_sent.aggregate_array('halite').getInfo()}),
                                                pd.DataFrame({'Date': GSL_NA_halite_sent.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer')

In [24]:
### Gypsum index collections ###
### Landsat
#GSl shoreline
GSL_NA_gypsum = combined_NA_NDWI_co_unclipped.map(GSL_NA_halite_watermask).map(MaskSaltPool).map(landsat_halite_mask).map(landsat_veg_mask).map(landsat_gypsum_fn).map(NorthArmClip).map(gypsum_pixel_count_landsat_GSLNA) 
GSL_SA_gypsum = combined_SA_NDWI_co_unclipped.map(GSL_SA_halite_watermask).map(FarmingtonBaySaltMask).map(landsat_veg_mask).map(landsat_halite_mask).map(landsat_gypsum_fn).map(SouthArmClip).map(gypsum_pixel_count_landsat_GSLSA)
BRB_gypsum = combined_BRB_NDWI_col_unclipped.map(BRB_halite_watermask).map(landsat_veg_mask).map(landsat_halite_mask).map(landsat_gypsum_fn).map(BRBClip).map(gypsum_pixel_count_landsat_BRB)

### Gypsum stats datatframe ###

landsat_gypsum_GSL_NA_df = pd.merge(pd.DataFrame({'Gypsum_Area': GSL_NA_gypsum.aggregate_array('gypsum').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer') 
landsat_gypsum_GSL_SA_df = pd.merge(pd.DataFrame({'Gypsum_Area': GSL_SA_gypsum.aggregate_array('gypsum').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')  
landsat_gypsum_BRB_df = pd.merge(pd.DataFrame({'Gypsum_Area': BRB_gypsum.aggregate_array('gypsum').getInfo()}),
                                                pd.DataFrame({'Date': BRB_landsat_dates}), right_index=True, left_index=True, how='outer') 

In [32]:
### Gypsum index collections ###
### Sentinel
#GSl shoreline
GSL_gypsum_sent = sent_GSL_NDWI_unclipped.map(GSL_S2_watermask).map(MaskSaltPool).map(FarmingtonBaySaltMask).map(sentinel_halite_mask).map(sentinel_veg_mask).map(sentinel_gypsum_fn).map(MaskSaltPool).map(TotalGSLClip)
GSL_NA_gypsum_sent = GSL_gypsum_sent.map(gypsum_pixel_count_sent_NA)
GSL_SA_gypsum_sent = GSL_gypsum_sent.map(gypsum_pixel_count_sent_SA)

### Gypsum stats datatframe ###

gypsum_GSL_NA_S2_df = pd.merge(pd.DataFrame({'Gypsum_Area': GSL_NA_gypsum_sent.aggregate_array('gypsum').getInfo()}),
                                                pd.DataFrame({'Date': GSL_NA_gypsum_sent.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer') 
gypsum_GSL_SA_S2_df = pd.merge(pd.DataFrame({'Gypsum_Area': GSL_SA_gypsum_sent.aggregate_array('gypsum').getInfo()}),
                                                pd.DataFrame({'Date': GSL_SA_gypsum_sent.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer') 

In [25]:
### Other sediments isolation ###
#Landsat
GSL_NA_carbonates = combined_NA_NDWI_co_unclipped.map(GSL_NA_halite_watermask).map(MaskSaltPool).map(landsat_veg_mask).map(MaskToCarbonatesL8).map(NorthArmClip).map(carbonate_pixel_count_landsat_GSLNA) #all seds except halite, gypsum, water, and vegetation, .map(MaskSaltPool)
GSL_SA_carbonates = combined_SA_NDWI_co_unclipped.map(GSL_SA_halite_watermask).map(MaskSaltPool).map(landsat_veg_mask).map(MaskToCarbonatesL8).map(SouthArmClip).map(carbonate_pixel_count_landsat_GSLSA)
BRB_carbonates = combined_BRB_NDWI_col_unclipped.map(BRB_halite_watermask).map(landsat_veg_mask).map(MaskToCarbonatesL8).map(BRBClip).map(carbonate_pixel_count_landsat_BRB)

### Carbs stats datatframe ###

landsat_carbonates_GSL_NA_df = pd.merge(pd.DataFrame({'Carb_muds_Area': GSL_NA_carbonates.aggregate_array('carb_muds').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer') 
landsat_carbonates_GSL_SA_df = pd.merge(pd.DataFrame({'Carb_muds_Area': GSL_SA_carbonates.aggregate_array('carb_muds').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer')  
landsat_carbonates_BRB_df = pd.merge(pd.DataFrame({'Carb_muds_Area': BRB_carbonates.aggregate_array('carb_muds').getInfo()}),
                                                pd.DataFrame({'Date': BRB_landsat_dates}), right_index=True, left_index=True, how='outer') 

In [33]:
### Other sediments isolation ###
#Sentinel
GSL_carbonates_sn = sent_GSL_NDWI_unclipped.map(GSL_S2_watermask).map(sentinel_veg_mask).map(MaskSaltPool).map(MaskToCarbonatesS2).map(TotalGSLClip)
GSL_NA_carbonates_sn = GSL_carbonates_sn.map(NorthArmClip).map(carbonate_pixel_count_sent_NA)
GSL_SA_carbonates_sn = GSL_carbonates_sn.map(SouthArmClip).map(carbonate_pixel_count_sent_SA)

### Carb muds stats datatframe ###

carbonates_GSL_NA_S2_df = pd.merge(pd.DataFrame({'Carb_muds_Area': GSL_NA_carbonates_sn.aggregate_array('carb_muds').getInfo()}),
                                                pd.DataFrame({'Date': GSL_NA_carbonates_sn.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer')
carbonates_GSL_SA_S2_df = pd.merge(pd.DataFrame({'Carb_muds_Area': GSL_SA_carbonates_sn.aggregate_array('carb_muds').getInfo()}),
                                                pd.DataFrame({'Date': GSL_SA_carbonates_sn.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer') 

In [20]:
### Chlorophyll / cyanobacteria detection ###
### Landsat Fluoresence Line Height adapted from Beck, R., Zhan, S., Liu, H., Tong, S., Yang, B., Xu, M., Ye, Z., Huang, Y., 
# Shu, S., Wu, Q., Wang, S., Berling, K., Murray, A., Emery, E., Reif, M., Harwood, J., Young, J., Nietch, C., Macke, D., … Su, H. (2016). 
# Comparison of satellite reflectance algorithms for estimating chlorophyll-a in a temperate reservoir using coincident hyperspectral 
#aircraft imagery and dense coincident surface observations. Remote Sensing of Environment, 178, 15–30. https://doi.org/10.1016/j.rse.2016.03.002

# GSL_NA_FLHV = combined_NA_NDWI_col.map(landsat_MaskToWater_NA).map(MaskSaltPool).map(landsat_KIVU).map(NorthArmClip).map(FLHV_pixel_count_landsat_GSLNA) #for landsat 5
# GSL_SA_FLHV = combined_SA_NDWI_col.map(landsat_MaskToWater_SA).map(landsat_KIVU).map(SouthArmClip).map(FLHV_pixel_count_landsat_GSLSA)

# GSL_NA_FLHV = combined_NA_NDWI_col.map(landsat_MaskToWater_NA).map(MaskSaltPool).map(landsat_FLHV).map(NorthArmClip).map(FLHV_pixel_count_landsat_GSLNA) #for landsat 8, need to figure out way to normalize results
# GSL_SA_FLHV = combined_SA_NDWI_col.map(landsat_MaskToWater_SA).map(landsat_FLHV).map(SouthArmClip).map(FLHV_pixel_count_landsat_GSLSA)

GSL_NA_avg_FLHV = combined_NA_NDWI_co_unclipped.map(landsat_MaskToWater_NA).map(MaskSaltPool).map(landsat_KIVU).map(NorthArmClip).map(FLHV_avg_landsat_GSLNA) #for getting average chla value for all images using the KIVU method
GSL_SA_avg_FLHV = combined_SA_NDWI_co_unclipped.map(landsat_MaskToWater_SA).map(landsat_KIVU).map(SouthArmClip).map(FLHV_avg_landsat_GSLSA)

### Algae stats datatframe ###
landsat_avg_algae_GSL_NA_df = pd.merge(pd.DataFrame({'Algae_Area': GSL_NA_avg_FLHV.aggregate_array('FLHV').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_NA_mosaic_col_dates}), right_index=True, left_index=True, how='outer') 
landsat_avg_algae_GSL_SA_df = pd.merge(pd.DataFrame({'Algae_Area': GSL_SA_avg_FLHV.aggregate_array('FLHV').getInfo()}),
                                                pd.DataFrame({'Date': landsat_GSL_mosaic_col_dates}), right_index=True, left_index=True, how='outer') 


In [21]:
### Chlorophyll / cyanobacteria detection ###
GSL_2BDA = sent_GSL_NDWI_unclipped.map(sentinel_MaskToWater).map(MaskSaltPool).map(sentinel_2BDA).map(TotalGSLClip)
GSL_2BDA_NA = GSL_2BDA.map(BDA_pixel_count_sent_NA).map(NorthArmClip)
GSL_2BDA_SA = GSL_2BDA.map(BDA_pixel_count_sent_SA).map(SouthArmClip)

GSL_2BDA_avg_NA = GSL_2BDA.map(BDA_average_sent_NA).map(NorthArmClip)
GSL_2BDA_avg_SA = GSL_2BDA.map(BDA_average_sent_SA).map(SouthArmClip)

### 2BDA cyanobacteria stats datatframe ###

algae_avg_GSL_NA_S2_df = pd.merge(pd.DataFrame({'Avg_Chl_a_2BDA': GSL_2BDA_avg_NA.aggregate_array('2BDA').getInfo()}),
                                                pd.DataFrame({'Date': GSL_2BDA_avg_NA.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer') 
algae_avg_GSL_SA_S2_df = pd.merge(pd.DataFrame({'Avg_Chl_a_2BDA': GSL_2BDA_avg_SA.aggregate_array('2BDA').getInfo()}),
                                                pd.DataFrame({'Date': GSL_2BDA_avg_NA.aggregate_array('Date_Filter').getInfo()}), right_index=True, left_index=True, how='outer')

In [34]:
### Save file center ###
#Uncomment needed lines

# landsat_GSL_NA_NDWI_threshold_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\landsat_ndwi_NA_thresholds_2014_2023.csv')
# landsat_GSL_NA_cloud_cover_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\landsat_NA_cloud_cover_2014_2023.csv')
# landsat_GSL_SA_NDWI_threshold_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\landsat_SA_ndwi_thresholds_2014_2023.csv')
# landsat_GSL_SA_cloud_cover_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\landsat_SA_cloud_cover_2014_2023.csv')
# sentinel_GSL_NA_cloud_cover_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Sentinel_GSL_NA_cloud_cover_2019_2020.csv')
# sentinel_GSL_SA_cloud_cover_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Sentinel_GSL_SA_cloud_cover_2019_2020.csv')

# landsat_halite_GSL_NA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Halite\landsat_halite_GSL_NA_2022_2023_update.csv')
# landsat_halite_GSL_SA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Halite\landsat_halite_GSL_SA_2022_2023_update.csv')
# landsat_gypsum_GSL_NA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Gypsum\landsat_gypsum_GSL_NA_2022_2023_update.csv')
# landsat_gypsum_GSL_SA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Gypsum\landsat_gypsum_GSL_SA_2022_2023_update.csv')
# landsat_carbonates_GSL_NA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Carbonate_muds\landsat_carbmuds_GSL_NA_2022_2023_update.csv')
# landsat_carbonates_GSL_SA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Carbonate_muds\landsat_carbmuds_GSL_SA_2022_2023_update.csv')
# landsat_GSLNA_ndwi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Water\landsat_water_GSL_NA_2022_2023_update.csv')
# landsat_GSLSA_ndwi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Water\landsat_water_GSL_SA_2022_2023_update.csv')
# landsat_algae_GSL_NA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Algae\landsat_algae_GSL_NA_2014_2023.csv')
# landsat_algae_GSL_SA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Algae\landsat_algae_GSL_SA_2014_2023.csv')
# landsat_avg_algae_GSL_NA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Algae\landsat_avg_algae_GSL_NA_2014_2023.csv')
# landsat_avg_algae_GSL_SA_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Algae\landsat_avg_algae_GSL_SA_2014_2023.csv')
# landsat_GSLSA_ndvi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Vegetation\landsat_NDVI_GSL_SA_2022_2023_update.csv')
# landsat_GSLSA_ndvi_S2Area_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Vegetation\landsat_NDVI_GSL_SA_S2Area_2022_2023_update.csv')
# landsat_GSLNA_ndvi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Vegetation\landsat_NDVI_GSL_NA_2022_2023_update.csv')
# landsat_GSLNA_LST_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\LST\landsat_NA_LST_14_23_mean.csv')
# landsat_GSLSA_LST_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\LST\landsat_SA_LST_14_23_mean.csv')
# landsat_GSL_USGS_LST_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\LST\landsat_USGS_LST_14_23_mean.csv')

# landsat_BRB_NDWI_threshold_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\landsat_ndwi_BRB_thresholds_1984_2023.csv')
# landsat_BRB_ndwi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Water\landsat_NDWI_BRB_1984_2023.csv')
# landsat_halite_BRB_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Halite\landsat_halite_BRB_1984_2023.csv')
# landsat_gypsum_BRB_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Gypsum\landsat_gypsum_BRB_1984_2023.csv')
# landsat_carbonates_BRB_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Carbonate_muds\landsat_carbs_BRB_1984_2023.csv')
# landsat_BRB_ndvi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Vegetation\landsat_NDVI_BRB_1984_2023.csv')

# sentinel_SA_halite_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Halite\sentinel_halite_GSL_SA_2022_2023_update.csv')
# sentinel_NA_halite_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Halite\sentinel_halite_GSL_NA_2022_2023_update.csv')
# gypsum_GSL_NA_S2_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Gypsum\sentinel_gypsum_GSL_NA_2022_2023_update.csv')
# gypsum_GSL_SA_S2_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Gypsum\sentinel_gypsum_GSL_SA_2022_2023_update.csv')
# carbonates_GSL_NA_S2_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Carbonate_muds\sentinel_carbmuds_GSL_NA_2022_2023_update.csv')
# carbonates_GSL_SA_S2_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Carbonate_muds\sentinel_carbmuds_GSL_SA_2022_2023_update.csv')
# sentinel_GSLNA_ndwi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Water\sentinel_water_GSL_NA_2022_2023_update.csv')
# sentinel_GSLSA_ndwi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Water\sentinel_water_GSL_SA_2022_2023_update.csv')
# algae_GSL_NA_S2_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Algae\sentinel_algae_GSL_NA_2019_2020.csv')
# algae_GSL_SA_S2_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Algae\sentinel_algae_GSL_SA_2019_2020.csv')
# algae_avg_GSL_NA_S2_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Algae\sentinel_avg_chla_2BDA_GSL_NA_2019_2023_v2.csv')
# algae_avg_GSL_SA_S2_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Algae\sentinel_avg_chla_2BDA_GSL_SA_2019_2023_v2.csv')
# sent_GSLNA_ndvi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Vegetation\sentinel_NDVI_GSL_NA_2022_2023_update.csv')
# sent_GSLSA_ndvi_df.to_csv('D:\\GSL Time Series Data 2022\GSL Shoreline Surface Types\Vegetation\sentinel_NDVI_GSL_SA_2022_2023_update.csv')


In [21]:
### Export file center ###
NA_export_col = landsat_GSL_NA_mosaic_col.select(['SR_B4', 'SR_B3', 'SR_B2']).map(NA_Export_Clip)
SA_export_col = landsat_GSL_mosaic_col.select(['SR_B4', 'SR_B3', 'SR_B2']).map(SA_Export_Clip)
NA_S2_export_col = sentinel_GSL_mosaic_col.select(['B4', 'B3', 'B2']).map(NA_Export_Clip)
SA_S2_export_col = sentinel_GSL_mosaic_col.select(['B4', 'B3', 'B2']).map(SA_Export_Clip)

### Export collections one at a time -- separated cells below for indivdual export calls ###

# geemap.ee_export_image_collection_to_drive(NA_export_col, descriptions=landsat_GSL_NA_mosaic_col_dates, folder='GSLNA_TrueColor_Images', scale=60, region=NA_export_region, skipEmptyTiles=True)
# geemap.ee_export_image_collection_to_drive(SA_export_col, descriptions=landsat_GSL_mosaic_col_dates, folder='GSLSA_TrueColor_Images', scale=60, region=SA_export_region, skipEmptyTiles=True)

# geemap.ee_export_image_collection_to_drive(NA_S2_export_col, descriptions=sentinel_GSL_mosaic_col_dates, folder='GSLNA_TrueColor_Images', scale=60, region=NA_export_region, skipEmptyTiles=True)
geemap.ee_export_image_collection_to_drive(SA_S2_export_col, descriptions=sentinel_GSL_mosaic_col_dates, folder='GSLSA_TrueColor_Images', scale=60, region=SA_export_region, skipEmptyTiles=True)

Total number of images: 81



In [28]:
geemap.ee_export_image_collection_to_drive(GSL_SA_NDWI, descriptions=landsat_GSL_mosaic_col_dates, folder='GSLSA_NDWI_Timeseries_1984_1994', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 70



In [29]:
geemap.ee_export_image_collection_to_drive(GSL_SA_NDVI, descriptions=landsat_GSL_mosaic_col_dates, folder='GSLSA_NDVI_Timeseries_1984_1994', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 70



In [30]:
geemap.ee_export_image_collection_to_drive(GSL_NA_NDVI, descriptions=landsat_GSL_NA_mosaic_col_dates, folder='GSLNA_NDVI_Timeseries_1984_1994', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 63



In [31]:
geemap.ee_export_image_collection_to_drive(GSL_SA_halite, descriptions=landsat_GSL_mosaic_col_dates, folder='GSLSA_halite_Timeseries_1984_1994', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 70



In [25]:
geemap.ee_export_image_collection_to_drive(GSL_NA_halite, descriptions=landsat_GSL_NA_mosaic_col_dates, folder='GSLNA_halite_Timeseries_1984_1994_saltpool', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 48



In [33]:
geemap.ee_export_image_collection_to_drive(GSL_SA_gypsum, descriptions=landsat_GSL_mosaic_col_dates, folder='GSLSA_gypsum_Timeseries_1984_1994', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 70



In [26]:
geemap.ee_export_image_collection_to_drive(GSL_NA_gypsum, descriptions=landsat_GSL_NA_mosaic_col_dates, folder='GSLNA_gypsum_Timeseries_1984_1994_saltpool', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 48



In [23]:
geemap.ee_export_image_collection_to_drive(GSL_SA_carbonates, descriptions=landsat_GSL_mosaic_col_dates, folder='GSLSA_carbonates_Timeseries_1984_1994', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 3



In [27]:
geemap.ee_export_image_collection_to_drive(GSL_NA_carbonates, descriptions=landsat_GSL_NA_mosaic_col_dates, folder='GSLNA_carbonates_Timeseries_1984_1994_saltpool', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 48



In [7]:
geemap.ee_export_image_collection_to_drive(GSL_NA_LST, descriptions=landsat_GSL_NA_mosaic_col_dates, folder='GSLNA_LST_14_23', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 63



In [8]:
geemap.ee_export_image_collection_to_drive(GSL_SA_LST, descriptions=landsat_GSL_mosaic_col_dates, folder='GSLSA_LST_14_23', scale=30, region=GSL_export_region, skipEmptyTiles=True)

Total number of images: 70

